In [ ]:
from numpy import array, dot, mean, std, empty, argsort
from numpy.linalg import eigh, solve
from numpy.random import randn
from matplotlib.pyplot import subplots, show
import matplotlib.pyplot as plt
% matplotlib inline

def cov(data):
    """
        covariance matrix
        note: specifically for mean-centered data
    """
    N = data.shape[1]
    C = empty((N, N))
    for j in range(N):
        C[j, j] = mean(data[:, j] * data[:, j])
        for k in range(N):
            C[j, k] = C[k, j] = mean(data[:, j] * data[:, k])
    return C

def pca(data, pc_count = None):
    """
        Principal component analysis using eigenvalues
        note: this mean-centers and auto-scales the data (in-place)
    """
    data -= mean(data, 0)
    data /= std(data, 0)
    C = cov(data)
    E, V = eigh(C)
    key = argsort(E)[::-1][:pc_count]
    E, V = E[key], V[:, key]
    U = dot(V.T, data.T).T
    return U, E, V

""" test data """
data = array([randn(8) for k in range(150)])
data[:50, 2:4] += 5
data[50:, 2:5] += 5

""" visualize """
trans = pca(data, 3)[0]
fig, (ax1, ax2) = subplots(1, 2)
ax1.scatter(data[:50, 0], data[:50, 1], c = 'r')
ax1.scatter(data[50:, 0], data[50:, 1], c = 'b')
ax2.scatter(trans[:50, 0], trans[:50, 1], c = 'r')
ax2.scatter(trans[50:, 0], trans[50:, 1], c = 'b')
show()
print(data.shape)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.zeros((50, 3))
x = np.arange(50)
data[:,0] = 2 * np.cos(0.5*x) + 0.1 * np.sin(x)
data[:,1] = 0.3 * np.sin(0.4*x) + 2 * np.cos(0.5*x)
data[:,2] = 2 * np.cos(0.5*x) + 3


result, e, v = pca(data)

In [ ]:
print(result.shape)
print(e.shape)
print(v.shape)
print(e)
print(v) # Looks like first row represents amplitudes for each eigenvector, required to reproduce the first row of the input data
plt.plot(data[:,0],'.')
plt.plot(data[:,1],'.')
plt.plot(data[:,2],'.')
plt.plot(result[:,0])
plt.plot(result[:,1])
plt.plot(result[:,2])
plt.show()

In [ ]:
# Load Alex compressive data
from analyze_dataset import *

frameRanges = None

# Analyze photodiode channels, with 4-point averaging to smooth out 50Hz interference
basePath = '/Users/jonny/Movies/Example_compr_sens_datasets/New datasets w jt phase annotation/Darkfield'
source='df_fish3_maskA_photodiodetifs_2000to3499'
pathFormat = '%s/'+source+'/%06d.tif'
(images, averagePeriod) = LoadAllImages(basePath+'/'+source, downsampleFactor=1, periodRange=None, plotAllPeriods=False, cropRect=None, timestampKey='time_processing_started')

# Apply rolling 4-point average for the images (to deal with noise on the raw compressive images)
if (True):
    for i in range(len(images)-3):
        images[i].image = (images[i].image + images[i+1].image + images[i+2].image + images[i+3].image) / 4.0

# Condense to a simple 2D array for PCA processing
signals = np.zeros((len(images)-3, 7))
print(images[i].image.shape)
for i in range(len(images)-3):
    signals[i,:] = images[i].image[:,0]

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)

plt.plot(signals[:,0]+1500,label='0')
plt.plot(signals[:,1]+500,label='1')
plt.plot(signals[:,2],label='2')
plt.plot(signals[:,3]-500,label='3')
plt.plot(signals[:,4],label='4')
plt.plot(signals[:,5]-1500,label='5')
plt.plot(signals[:,6],label='6')
plt.legend()
plt.show()

In [ ]:
result, e, v = pca(signals)

In [ ]:
m=500
plt.plot(result[:m,0]/6+7,label='0')
plt.plot(result[:m,1]/4+5.5,label='1')
plt.plot(result[:m,2]/2+4.5,label='2')
plt.plot(result[:m,3]+2.8,label='3')
plt.plot(result[:m,4]+1.4,label='4')
plt.plot(result[:m,5]+0.6,label='5')
plt.plot(result[:m,6],label='6')
plt.legend()
plt.title('Principal components')
plt.show()

In [ ]:
# Do phase analysis of Alex's compressive data, so we can generate an estimate of a smoothed noise-reduced single period
# The intention is to use the phases from the widefield analysis to assign known phases to the photodiode channels

from analyze_dataset import *

frameRanges = None

if True:
    # Analyze photodiode channels, with 4-point averaging to smooth out 50Hz interference
    # Note that we look for time_processing_started in the photodiode channel. That is just a synthetic timestamp, and I presume that's the key I suggested Alex fills in.
    basePath = '/Users/jonny/Movies/Example_compr_sens_datasets/New datasets w jt phase annotation/Darkfield'
    (kt1, kp1, globalShiftSolution, res, shifts, sequenceDrifts) = AnalyzeDataset(basePath, frameRanges, ['df_fish3_maskA_photodiodetifs_2000to3499'], periodRange = np.arange(100, 140, 0.2), numSamplesPerPeriod = 200, source='df_fish3_maskA_photodiodetifs_2000to3499', applyDriftCorrection=False, downsampling=1, interpolationDistanceBetweenSequences=4, rollingAverage=True, sourceTimestampKey='time_processing_started', fluorTimestampKey='time_processing_started')
    
if True:
    basePath = '/Users/jonny/Movies/Example_compr_sens_datasets/New datasets w jt phase annotation/Darkfield'
    # NOTE that I have set downsampling to 2 for this script, so processing isn't too ridiculously slow.
    (kt3, kp3, globalShiftSolution, res, shifts, sequenceDrifts) = AnalyzeDataset(basePath, frameRanges, ['df_fish3_maskA_widefield_800to1399'], periodRange = np.arange(30, 56, 0.1), numSamplesPerPeriod = 80, source='df_fish3_maskA_widefield_800to1399', applyDriftCorrection=False, downsampling=2, interpolationDistanceBetweenSequences=4, sourceTimestampKey='synthetic_timestamp', fluorTimestampKey='synthetic_timestamp')

In [ ]:
# Use the standard phase-assignment code to assign phases to the photodiode channel,
# using the phases calculated from the brightfield channel
#def AnnotateDatasetUsingBrightfieldPhases(basePath, imageRangeList, fluorFolders=[], source='Brightfield - Prosilica', key='phase_from_offline_sync_analysis'):

# images was already populated with the (4-point-smoothed) photodiode signals earlier
print(images.size)
print('First image', os.path.basename(images[0].path))
print('Last image', os.path.basename(images[-1].path))
InterpolateForImagePhases(images, kt3/5, kp3)

phaseAndValues = np.zeros((images.size,8))
for i in range(images.size):
    phaseAndValues[i,0] = images[i].phase
    phaseAndValues[i,1:8] = images[i].image[:,0]
sortedPhaseAndValues = phaseAndValues[np.argsort(phaseAndValues[:,0])]
sortedPhaseAndValues = sortedPhaseAndValues[0:1398,:]

In [ ]:
old = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = (3,6)


def runningMeanFast(x, N):
#    return np.convolve(x, np.ones((N,))/N)[(N-1):]
    # this behaves in a correct circular manner
    return scipy.ndimage.filters.convolve1d(x, np.ones((N,))/N, mode='wrap')

def GaussianMean(x, w):
    # Calculate a Gaussian-weighted mean with a width of w, based on the actual phase
    # differences between each sample
    # x is an Nx8 matrix, where the first column has the phase values
    result = x.copy()
    for i in range(x.shape[0]):
        # Calculate the Gaussian-weighted value for entry i in the input matrix
        # First calculate the weightings
        phaseDiffs = np.abs(x[:,0] - x[i,0])
        wrapped = np.where(phaseDiffs > np.pi)
        phaseDiffs[wrapped] = 2.0 * np.pi - phaseDiffs[wrapped]
        weightings = np.exp(-(phaseDiffs/w)**2)
        weightings = weightings / np.sum(weightings)
        # Now apply these weightings to the photodiode values
        for j in range(1, 8):
            result[i,j] = np.sum(x[:,j]*weightings)
    return result
            

# Plot the signals to check that they are similar to the original unsmoothed signals.
plt.xlim([0, 2*np.pi])
plt.ylim([0, 8000])

if False:
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,1]+1500, 20))
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,2]+500, 20))
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,3], 20))
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,4]-500, 20))
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,5], 20))
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,6]-1500, 20))
    plt.plot(sortedPhaseAndValues[:,0], runningMeanFast(sortedPhaseAndValues[:,7], 20))
else:
    gaussianSmoothed = GaussianMean(sortedPhaseAndValues, 0.05)#0.09)
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,1]+1500)
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,2]+500)
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,3])
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,4]-500)
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,5])
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,6]-1500)
    plt.plot(gaussianSmoothed[:,0], gaussianSmoothed[:,7])

plt.rcParams["figure.figsize"] = old


In [ ]:
# Calculate smoothed profiles for all channels, and do PCA on them
if False:
    smoothedSignals = np.zeros((sortedPhaseAndValues.shape[0], 7))
    for i in range(7):
        smoothedSignals[:,i] = runningMeanFast(sortedPhaseAndValues[:,1+i], 20)
else:
    smoothedSignals = gaussianSmoothed[:,1:8]
print(smoothedSignals.shape)

In [ ]:
result, e, v = pca(smoothedSignals[:,:])

In [ ]:
m=500
plt.xlim([0, 20])#2*np.pi])
if False:
    # Separate out the components for display purposes
    plt.plot(sortedPhaseAndValues[:,0], result[:,0]/6+7,label='0')
    plt.plot(sortedPhaseAndValues[:,0], result[:,1]/4+5.5,label='1')
    plt.plot(sortedPhaseAndValues[:,0], result[:,2]/2+4.5,label='2')
    plt.plot(sortedPhaseAndValues[:,0], result[:,3]+2.8,label='3')
    plt.plot(sortedPhaseAndValues[:,0], result[:,4]+1.4,label='4')
    plt.plot(sortedPhaseAndValues[:,0], result[:,5]+0.6,label='5')
    plt.plot(sortedPhaseAndValues[:,0], result[:,6],label='6')
else:
    # Plot the components on a common axis
    plt.plot(sortedPhaseAndValues[:,0], result[:,0],label='0')
    plt.plot(sortedPhaseAndValues[:,0], result[:,1],label='1')
    plt.plot(sortedPhaseAndValues[:,0], result[:,2],label='2')
    plt.plot(sortedPhaseAndValues[:,0], result[:,3],label='3')
    plt.plot(sortedPhaseAndValues[:,0], result[:,4],label='4')
    plt.plot(sortedPhaseAndValues[:,0], result[:,5],label='5')
    plt.plot(sortedPhaseAndValues[:,0], result[:,6],label='6')

plt.legend()
plt.show()

In [ ]:
# ****** The PCA mean-centers the data with unit std.
# What does this mean in terms of the eigenvalues? 
# I think it means that the PCA results reconstruct this normalized signal, not the original signal.
# Confirm that I am correct, by plotting graphs
recoveredSignals = np.zeros(result.shape)
channelToPlot = 4

for channel in range(7):#channelToPlot,channelToPlot+1): # Make sure I set this to 7 in order to reconstruct all channels
    #print('channel %d' % channel)
    plt.plot(sortedPhaseAndValues[:,channel+1])
    for eigenvalue in range(7):# Make sure I set this to 7 in order to reconstruct full signal
        recoveredSignals[:,channel] = recoveredSignals[:,channel] + v[channel,eigenvalue]*result[:,eigenvalue]
        print(' ampl %d: %f' % (eigenvalue, v[channel,eigenvalue]))
        if eigenvalue >= 6:
            plt.plot(recoveredSignals[:,channel] * np.std(sortedPhaseAndValues[:,channel+1]) + np.mean(sortedPhaseAndValues[:,channel+1]), linewidth=3, color='red')
    plt.show()
#plt.plot(recoveredSignals[:,channelToPlot] * np.std(sortedPhaseAndValues[:,channelToPlot+1]) + np.mean(sortedPhaseAndValues[:,channelToPlot+1]))

# Visual estimate of how many eigenfunctions are required to reproduce the input signal convincingly
# (starting from the first one each time, rather than cherry-picking, of course)
# "Noise" is the number of channels that are very clearly separated from the noise in this plot
# Channel   Good    Near-perfect  Noise
# 0         2       3-7           3
# 1         3       4             2
# 2         4       4             2
# 3         3       4+            3
# 4         3       5             4
# 5         3       3             2
# 6         2       2             1-2
#
# This would seem to suggest that we have about 4 significant independent modes in our signals.
# It also suggests that the first and last channel are fairly redundant in their information?
#
# I still need to work out how to interpret all this information, though.
# I think I need some sense of where the "noise floor" is, and what that means for the useful information 
# I can extract from the signals. Thanks to channel #4, it looks like I can definitely say that 4 modes 
# are above the noise floor, but it would be nice to be a bit more rigourous about this.


In [ ]:
# ***** I need to think about how much of the total energy is represented in each eigenfunction,
# and whether it matters whether or not the signals are normalized.
# For my SAD, the offset of an individual channel makes no difference.
# The scaling (STD) does make a difference in that one higher-amplitude signal will tend to dominate.
# However, a low amplitude signal that will still contribute useful information.
# I definitely think the noise floor needs to be taken into consideration here, though I am not sure how.

# I could state that x% of the energy of the smoothed signals can be reproduced using N principal components.
# I think that statement is valid for any ONE of the smoothed signals, regardless of the normalization used.
# (although I still need to think more about what numbers in the analysis to look at to be able to make that statement!)
# However, I need to think more about how to make such a statement about ALL the signals together, in a meaningful way.

# Looking at the reconstruction from the first N eigenfunctions (see above), it does seem to genuinely be the case that
# there are subtle but real features in the curves that can only be reproduced if we use all 7 eigenfunctions.
print(e)
print(np.cumsum(e)/np.sum(e))
print(v)